# Import Lib

In [1]:
import cv2
from facenet_pytorch import MTCNN#https://github.com/timesler/facenet-pytorch
import torch
import time
import os
import numpy as np
import shutil
import math
import glob
import torch 
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization
import os
from PIL import Image
import numpy as np
import random
device =  torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

C:\Users\ADMIN\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


# Face Detection

In [2]:
mtcnn = MTCNN(thresholds= [0.7, 0.7, 0.8],min_face_size=80,keep_all=True, device = device)#Thresholds is for 3 nets P, R and O. Defaut value are [0.6, 0.7, 0.7] 
#Keep_all to let model know if we want to detect and return all the face in the capture

In [3]:
WIDTH = 640#size of capture
HEIGHT = 480
prev_frame_time = 0
new_frame_time = 0

In [4]:

# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture("C:\\Users\\ADMIN\\Documents\\GitHub\\FaceRecognition-empty\\with pretrained model\\data\\vid\\Team.mp4")
cap.set(cv2.CAP_PROP_FRAME_WIDTH,WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,HEIGHT)
while cap.isOpened():
    isSuccess, frame = cap.read()
    if isSuccess==False : break
    if isSuccess:
        start_time = time.time()
        boxes, _ = mtcnn.detect(frame)
        end_time = time.time()
        if boxes is not None:
            for box in boxes:
                print(box)
                bbox = list(map(int,box.tolist()))
                frame = cv2.rectangle(frame,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(0,0,255),6)
                
                frame = cv2.putText(frame,str(int(1/(end_time-start_time))),(0, 25),cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 255, 0),2,cv2.LINE_AA, False)
                cv2.putText(frame, str(bbox), (bbox[0],bbox[1]), cv2.FONT_HERSHEY_DUPLEX, 1, (100, 255, 0), 3, cv2.LINE_AA)
                time.sleep(0.0000000000001)
    cv2.imshow('Face Detection', frame)
    if cv2.waitKey(1)&0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()

[652.8506469726562 233.7052001953125 856.620849609375 491.2525634765625]
[223.8394012451172 236.81600952148438 405.4974365234375 457.4383239746094]
[1008.12109375 103.744140625 1119.513427734375 233.1702423095703]
[653.5598754882812 234.71237182617188 856.4798583984375 490.90655517578125]
[225.12283325195312 235.51577758789062 407.8699951171875
 458.13665771484375]
[1007.7916259765625 103.9172592163086 1119.42626953125 232.6063995361328]
[658.7639770507812 240.62796020507812 854.6891479492188 486.5045471191406]
[223.78013610839844 239.18914794921875 405.9502258300781 461.126220703125]
[1006.7545776367188 101.91900634765625 1121.93359375 236.71133422851562]
[652.4445190429688 240.82354736328125 846.8778686523438 491.35931396484375]
[224.91371154785156 238.76605224609375 407.5829772949219 461.8937683105469]
[657.0046997070312 240.74903869628906 858.2183227539062 490.1163024902344]
[225.4509735107422 239.22943115234375 407.78302001953125 461.2403259277344]
[656.3372802734375 238.781005859

# Capture some frames to prepare for inference

In [5]:
IMG_PATH = r'./data/img/'
VID_SRC = r'./data/vid/'

In [6]:
usr_name = input("Input ur name: ")
if(usr_name!=''):
    USR_PATH = os.path.join(IMG_PATH, usr_name)
    Source = input("Source: ")
    if(Source=='cam'):
        cap = cv2.VideoCapture(0)
    else : 
        VID_PATH = os.path.join(VID_SRC, usr_name+'.mp4')
        cap = cv2.VideoCapture(VID_PATH)
    if os.path.exists(USR_PATH):
        shutil.rmtree(USR_PATH)
    os.makedirs(USR_PATH)
    from datetime import datetime

    mtcnn = MTCNN(margin = 20,min_face_size=80, thresholds= [0.7, 0.7, 0.8],keep_all=False, select_largest = True, post_process=False, device = device)

    cap.set(cv2.CAP_PROP_FRAME_WIDTH,WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT,HEIGHT)
    while cap.isOpened():
        isSuccess, frame = cap.read()
        if(isSuccess ==False):break
        if isSuccess:
            boxes, _ = mtcnn.detect(frame)
            if boxes is not None:
                for box in boxes:
                        bbox = list(map(int,box.tolist()))
                        dst_points = np.float32([[bbox[0], bbox[1]], [bbox[2], bbox[1]],  [bbox[2], bbox[3]],[bbox[0], bbox[3]]])
                        src_points = np.float32([[0, 0],             [WIDTH, 0],          [WIDTH, HEIGHT],   [0, HEIGHT]])
                        perspective_matrix = cv2.getPerspectiveTransform(dst_points,src_points)
                        warped_image = cv2.warpPerspective(frame, perspective_matrix, (WIDTH, HEIGHT))
                        path = str(USR_PATH+'/{}.jpg'.format(str(datetime.now())[:-7].replace(":","-").replace(" ","-")+str(random.randint(1, 999))))
                        frame = cv2.rectangle(frame,(bbox[0],bbox[1]),(bbox[2],bbox[3]),(0,0,255))# bbox[0]:top_left_x,bbox[1]:top_left_y,bbox[2]:down_right_x,bbox[3]:down_right_y
                        cv2.imwrite(path, warped_image)
        cv2.imshow('Face Detection', frame)
        if cv2.waitKey(1)&0xFF == 27:
            break
        
    cap.release()
    cv2.destroyAllWindows()

# Face Recognition

In [7]:
IMG_PATH = './data/img'
DATA_PATH = './data'
embeddings = []
names = []

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [8]:
def trans(img):
    transform = transforms.Compose([
            np.float32, 
            transforms.ToTensor(),
            fixed_image_standardization
        ])
    return transform(img)

In [9]:
def fixed_image_standardization(image_tensor):
    processed_tensor = (image_tensor - 127.5) / 128.0
    return processed_tensor

In [10]:
from torch import nn
class ModifiedModel(nn.Module):
    def __init__(self, model):
        super(ModifiedModel, self).__init__()
        self.model = model  # Load the pre-trained model

        # Add a new Softmax layer with dim=4
        self.softmax_layer = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.model(x)  # Pass the input through the pre-trained model
        x = x.view(x.size(0), -1)  # Flatten the tensor
        output = self.softmax_layer(x)
        return output
model=ModifiedModel(InceptionResnetV1(
    classify=False,
    pretrained="vggface2",dropout_prob=0.9).to(device))
model.eval()

ModifiedModel(
  (model): InceptionResnetV1(
    (conv2d_1a): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (conv2d_2a): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (conv2d_2b): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2d_3b): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, 

In [11]:
embeddings = []
names = []
for usr in os.listdir(IMG_PATH):
    embeds = []
    for file in glob.glob(os.path.join(IMG_PATH, usr)+'/*.jpg'):
        try:
            img = Image.open(file)
        except:
            continue
        with torch.no_grad():
            embeds.append(model(trans(img).to(device).unsqueeze(0))) #1 anh, kich thuoc [1,512]
    if len(embeds) == 0:
        continue
    embedding = torch.cat(embeds).mean(0, keepdim=True) #dua ra trung binh cua 50 anh, kich thuoc [1,512]
    embeddings.append(embedding) # 1 cai list n cai [1,512]
    names.append(usr)

In [12]:
embeddings = torch.cat(list(embeddings)) #[n,512]
names = np.array(names)
torch.save(embeddings, DATA_PATH+"/faceslist.pth")
np.save(DATA_PATH+"/usernames", names)

In [13]:
embeddings.tolist()

[[0.0021156403236091137,
  0.001921648858115077,
  0.001883369404822588,
  0.0020845846738666296,
  0.0018200103659182787,
  0.001800074940547347,
  0.0018278133356943727,
  0.002048791153356433,
  0.0019910777918994427,
  0.0019976207986474037,
  0.0019901159685105085,
  0.0020258885342627764,
  0.0018021631985902786,
  0.0018853263463824987,
  0.001988970674574375,
  0.001849622931331396,
  0.0019123669480904937,
  0.0021085513290017843,
  0.0019564004614949226,
  0.002084001898765564,
  0.001960485242307186,
  0.002018261468037963,
  0.0018985597416758537,
  0.001798245357349515,
  0.001770803239196539,
  0.001929844031110406,
  0.0019483949290588498,
  0.0019778553396463394,
  0.0019088583067059517,
  0.002047198126092553,
  0.0020361263304948807,
  0.001955895684659481,
  0.002036493504419923,
  0.0019731379579752684,
  0.0017911304021254182,
  0.00186274538282305,
  0.0018824503058567643,
  0.0020648513454943895,
  0.002092694863677025,
  0.0020804088562726974,
  0.00192415504716

In [14]:
def load_faceslist():
    embeds = torch.load(DATA_PATH+'/faceslist.pth')
    names = np.load(DATA_PATH+'/usernames.npy')
    print(embeds, names)
    return embeds, names

In [15]:
def inference(model, face, local_embeds, threshold = 0.7,power = pow(10, 2)):
    #local: [n,512] voi n la so nguoi trong faceslist
    embeds = []
    # print(trans(face).unsqueeze(0).shape)
    embeds.append(model(trans(face).to(device).unsqueeze(0)))
    detect_embeds = torch.cat(embeds) #[1,512]
    print(detect_embeds.shape)
    # print(detect_embeds.shape)
                    #[1,512,1]                                      [1,512,n]
    norm_diff = detect_embeds.unsqueeze(-1) - torch.transpose(local_embeds, 0, 1).unsqueeze(0)
    # print(norm_diff)
    norm_score = torch.sqrt(torch.sum(torch.pow(norm_diff, 2), dim=1)).tolist()[0] #(1,n), moi cot la tong khoang cach euclide so vs embed moi

    min_dist, embed_idx = min(norm_score), norm_score.index(min(norm_score))
    
    # print(min_dist.shape)
    if min_dist*power > threshold:
        return -1, -1
    else:
        return embed_idx, min_dist

In [16]:
def extract_face(box, img, margin=20):
    face_size = 160
    img_size = (WIDTH,HEIGHT)
    margin = [
        margin * (box[2] - box[0]) / (face_size - margin),
        margin * (box[3] - box[1]) / (face_size - margin),
    ] 

    box = [ #box[0] và box[1] là tọa độ của điểm góc trên cùng trái
        int(max(box[0] - margin[0] / 2, 0)), #nếu thêm vào margin bị ra khỏi rìa ảnh => đưa về điểm 0
        int(max(box[1] - margin[1] / 2, 0)),
        int(min(box[2] + margin[0] / 2, img_size[0])), #nếu thêm vào margin bị ra khỏi rìa ảnh => đưa về tọa độ của ảnh gốc
        int(min(box[3] + margin[1] / 2, img_size[1])),
        
    ] #tạo margin mới bao quanh box cũ
    # img = img[box[1]:box[3], box[0]:box[2]]
    dst_points = np.float32([[box[0], box[1]], [box[2], box[1]],  [box[2], box[3]],[box[0], box[3]]])
    src_points = np.float32([[0, 0],             [WIDTH, 0],          [WIDTH, HEIGHT],   [0, HEIGHT]])

    perspective_matrix = cv2.getPerspectiveTransform(dst_points,src_points)
    img = cv2.warpPerspective(img, perspective_matrix, (WIDTH, HEIGHT))
    face = cv2.resize(img,(face_size, face_size), interpolation=cv2.INTER_AREA)
    face = Image.fromarray(face)
    return face

In [21]:
embeddings, names = load_faceslist()
print(embeddings, names )
power = pow(10,2)
mtcnn = MTCNN(min_face_size=80, keep_all=True,thresholds= [0.7, 0.7, 0.8], select_largest = True, post_process=False, device = device)

VID_SRC = r'./data/vid/'
Source = input("Source: ")
if(Source=='cam'):
    cap = cv2.VideoCapture(0)
else : 
    VID_PATH = os.path.join(VID_SRC, Source+'.mp4')
    cap = cv2.VideoCapture(VID_PATH)

cap.set(cv2.CAP_PROP_FRAME_WIDTH,WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,HEIGHT)
while cap.isOpened():
        isSuccess, frame = cap.read()
        if isSuccess ==False:break
        else:
            boxes, _ = mtcnn.detect(frame)
            if boxes is not None:
                for box in boxes:
                    bbox = list(map(int,box.tolist()))
                    face = extract_face(bbox, frame)
                    idx, score = inference(model, face, embeddings)
                    if idx != -1:
                        frame = cv2.rectangle(frame, (bbox[0],bbox[1]), (bbox[2],bbox[3]), (0,0,255), 6)
                        score = score*power
                        frame = cv2.putText(frame, names[idx] + '_{:.2f}'.format(score), (bbox[0],bbox[1]), cv2.FONT_HERSHEY_DUPLEX, 2, (0,255,0), 2, cv2.LINE_8)
                    else:
                        frame = cv2.rectangle(frame, (bbox[0],bbox[1]), (bbox[2],bbox[3]), (0,0,255), 6)
                        frame = cv2.putText(frame,'Unknown', (bbox[0],bbox[1]), cv2.FONT_HERSHEY_DUPLEX, 2, (0,255,0), 2, cv2.LINE_8)
            new_frame_time = time.time()
            fps = 1/(new_frame_time-prev_frame_time)
            prev_frame_time = new_frame_time
            fps = str(int(fps))
            cv2.putText(frame, fps, (7, 70), cv2.FONT_HERSHEY_DUPLEX, 3, (100, 255, 0), 3, cv2.LINE_AA)
        cv2.imshow('Face Recognition', frame)
        if cv2.waitKey(1)&0xFF == 27:
            break
    
cap.release()
cv2.destroyAllWindows()

tensor([[0.0021, 0.0019, 0.0019,  ..., 0.0019, 0.0020, 0.0019],
        [0.0021, 0.0019, 0.0019,  ..., 0.0019, 0.0020, 0.0019],
        [0.0020, 0.0019, 0.0019,  ..., 0.0018, 0.0019, 0.0019],
        [0.0021, 0.0019, 0.0019,  ..., 0.0019, 0.0021, 0.0019]],
       device='cuda:0') ['Gia' 'Hoang' 'Nam' 'Tuan']
tensor([[0.0021, 0.0019, 0.0019,  ..., 0.0019, 0.0020, 0.0019],
        [0.0021, 0.0019, 0.0019,  ..., 0.0019, 0.0020, 0.0019],
        [0.0020, 0.0019, 0.0019,  ..., 0.0018, 0.0019, 0.0019],
        [0.0021, 0.0019, 0.0019,  ..., 0.0019, 0.0021, 0.0019]],
       device='cuda:0') ['Gia' 'Hoang' 'Nam' 'Tuan']
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1, 512])
to